### NN1

In [1]:
import numpy as np
import pandas as pd

In [254]:
class NeuralNetwork:
    
    def __init__(self, layers, weights=None, biases=None, activations=None):
        self.layers = layers
        self.num_layers = len(layers)
        
        if weights is None:
            self.weights = [np.random.randn(layers[i-1], layers[i]) for i in range(1, self.num_layers)]
        else:
            self.weights = weights
        
        if biases is None:
            self.biases = [np.random.randn(1, layers[i]) for i in range(1, self.num_layers)]
        else:
            self.biases = biases
        
        if activations is None:
            self.activations = ['sigmoid' for i in range(self.num_layers)]
        else:
            self.activations = activations
        
        activation_functions = {
            'sigmoid': self._sigmoid,
            'relu': self._relu,
            'linear': self._linear
        }
        self.activation_funcs = list(map(lambda x: activation_functions.get(x), self.activations))

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def _relu(self, z):
        return np.maximum(0, z)

    def _linear(self, z):
        return z
    
    def feedforward(self, a):
        for w, b, func in zip(self.weights, self.biases, self.activation_funcs):
#             print(a)
#             print(w)
#             print(b)
            z = np.dot(a, w) + b
            a = func(z)
        return a
    
    def predict(self, df):
        return df.apply(lambda x: self.feedforward(np.array([x]))[0][0])

1/5

In [247]:
weights_1 = [np.random.randn(1, 5), np.random.randn(5, 1)]
biases_1 = [np.random.randn(1, 5), np.random.randn(1, 1)] 
activations_1 = ['sigmoid', 'linear']

In [248]:
nn_1 = NeuralNetwork([1, 5, 1], weights=weights_1, biases=biases_1, activations=activations_1)

In [238]:
nn_1.feedforward(np.array([1]))

array([[2.51197147]])

1/10

In [209]:
weights_2 = [np.random.randn(1, 10), np.random.randn(10, 1)]
biases_2 = [np.random.randn(1, 10), np.random.randn(1, 1)] 
activations_2 = ['sigmoid', 'linear']

In [210]:
nn_2 = NeuralNetwork([1, 10, 1], weights=weights_2, biases=biases_2, activations=activations_2)

In [211]:
nn_2.feedforward(np.array([1]))

array([[-2.46340581]])

2/5

In [215]:
weights_3 = [np.random.randn(1, 5), np.random.randn(5, 5), np.random.randn(5, 1)]
biases_3 = [np.random.randn(1, 5), np.random.randn(1, 5), np.random.randn(1, 1)] 
activations_3 = ['sigmoid', 'sigmoid', 'linear']

In [216]:
nn_3 = NeuralNetwork([1, 5, 5, 1], weights=weights_3, biases=biases_3, activations=activations_3)

In [217]:
nn_3.feedforward(np.array([1]))

array([[1.25845247]])

### TEST

In [227]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [221]:
square_simple = pd.read_csv("dane/regression/square-simple-test.csv", index_col=0)
steps_large = pd.read_csv("dane/regression/steps-large-test.csv", index_col=0)

In [228]:
y_pred = nn_1.predict(square_simple['x'])
MSE(square_simple['y'], y_pred)

8698.929821521371

In [229]:
min_mse = 10000
min_weights = None
min_biases = None
for i in range(100000):
    weights_1 = [np.random.randn(1, 5), np.random.randn(5, 1)]
    biases_1 = [np.random.randn(1, 5), np.random.randn(1, 1)] 
    activations_1 = ['sigmoid', 'linear']
    nn_1 = NeuralNetwork([1, 5, 1], weights=weights_1, biases=biases_1, activations=activations_1)
    y_pred = nn_1.predict(square_simple['x'])
    mse = MSE(square_simple['y'], y_pred)
    if mse < min_mse:
        min_mse = mse
        min_weights = weights_1
        min_biases = biases_1

In [230]:
min_mse

8157.45203671807

In [223]:
y_pred = nn_2.predict(square_simple['x'])
mse(square_simple['y'], y_pred)

8547.47430687707

In [224]:
y_pred = nn_3.predict(square_simple['x'])
mse(square_simple['y'], y_pred)

8784.95126512737

In [225]:
y_pred = nn_1.predict(steps_large['x'])
mse(steps_large['y'], y_pred)

7603.799698483237

### tensor